In [1]:
## โค้ดนี้พัฒนาจาก AC-aggwithWindow-withOrderedWindow.ipynb

In [2]:
# Necessary to change the Kafka broker's IP number to the INTERNAL IP Address and port 9092

kafka_broker = "10.128.0.12:9092"

In [3]:
# Configuring the settings for the Spark job

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,com.microsoft.azure:spark-mssql-connector:1.0.2").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.microsoft.azure#spark-mssql-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75564d47-8b6a-486a-bc93-da8d0173b078;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in 

In [4]:
# Creating a dataframe from event stream coming from a Kafka topic

rawMetadata_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_broker) \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [5]:
# Importing Spark functions

from pyspark.sql import functions as sparkf

In [6]:
# Printing the schema of the raw data

rawMetadata_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Selecting only key and value columns and converting their data type to string

onlyMetadata_df = rawMetadata_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [8]:
# Defining a schema for the incoming JSON data

from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

schema = StructType([
    StructField("locationId", StringType()),
    StructField("location", StringType()),
    StructField("parameter", StringType()),
    StructField("value", StringType()),
    StructField("date", StringType()),
    StructField("unit", StringType()),
    StructField("coordinates", StringType()),
    StructField("country", StringType()),
    StructField("city", StringType()),
    StructField("isMobile", StringType()),
    StructField("isAnalysis", StringType()),
    StructField("entity", StringType()),
    StructField("sensorType", StringType())
])


# Nested 'date' and 'coordinates' schema

date_schema = StructType([
    StructField("utc", StringType()),
    StructField("local", StringType())
])

coordinates_schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType())
])


# Parsing the JSON string column and converting it to a struct

parsedData_df = onlyMetadata_df.withColumn("data", from_json("value", schema))

In [9]:
parsedData_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- isMobile: string (nullable = true)
 |    |-- isAnalysis: string (nullable = true)
 |    |-- entity: string (nullable = true)
 |    |-- sensorType: string (nullable = true)



In [10]:
# Selecting only the nested fields

unNested_df = parsedData_df.select("key", "data.*")

In [11]:
unNested_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [12]:
# Testing for Extraction of 'date' and 'coordinates' fields from the nested structure

unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: st

In [13]:
# Extracting 'date' and 'coordinates' fields from the nested structure

extractedDateLatLong_df = unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.select('key',
 'locationId',
 'location',
 'parameter',
 'value',
 'date.*',
 'unit',
 'coordinates.*',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType')

In [14]:
extractedDateLatLong_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [15]:
# Converting 'utc' column to timestamp and 'value' column to float

from pyspark.sql.types import *

final_df = extractedDateLatLong_df\
.withColumn('sourceSTP',sparkf.to_timestamp(sparkf.col('utc'), "yyyy-MM-dd'T'HH:mm:ssXXX"))\
.withColumn('value',sparkf.col('value').cast(FloatType()))

In [16]:
final_df.columns

['key',
 'locationId',
 'location',
 'parameter',
 'value',
 'utc',
 'local',
 'unit',
 'latitude',
 'longitude',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType',
 'sourceSTP']

In [17]:
final_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: float (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- sourceSTP: timestamp (nullable = true)



In [18]:
from pyspark.sql import functions as F
from pyspark.sql.functions import window

windowedCounts = final_df.groupBy(
    window(final_df.sourceSTP, "360 minutes", "360 minutes"),
    final_df.value
).count()

orderwindowedCounts = windowedCounts.orderBy('window','value')

query = orderwindowedCounts.writeStream.queryName("aggWindow").outputMode("complete").format("memory").start()
query.awaitTermination(60)


23/06/22 16:18:32 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-be2f5120-25d7-4f72-a073-6186c81210cf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


False

In [19]:
spark.sql('select * from aggWindow').show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |value|count|
+------------------------------------------+-----+-----+
|[2022-12-31 18:00:00, 2023-01-01 00:00:00]|36.0 |6    |
|[2023-01-01 00:00:00, 2023-01-01 06:00:00]|30.0 |2    |
|[2023-01-01 00:00:00, 2023-01-01 06:00:00]|31.0 |2    |
|[2023-01-01 00:00:00, 2023-01-01 06:00:00]|33.0 |1    |
|[2023-01-01 00:00:00, 2023-01-01 06:00:00]|34.0 |1    |
|[2023-01-01 06:00:00, 2023-01-01 12:00:00]|29.0 |6    |
|[2023-01-01 12:00:00, 2023-01-01 18:00:00]|29.0 |2    |
|[2023-01-01 12:00:00, 2023-01-01 18:00:00]|30.0 |4    |
|[2023-01-01 18:00:00, 2023-01-02 00:00:00]|31.0 |4    |
|[2023-01-01 18:00:00, 2023-01-02 00:00:00]|32.0 |2    |
|[2023-01-02 00:00:00, 2023-01-02 06:00:00]|33.0 |2    |
|[2023-01-02 00:00:00, 2023-01-02 06:00:00]|34.0 |1    |
|[2023-01-02 00:00:00, 2023-01-02 06:00:00]|35.0 |2    |
|[2023-01-02 00:00:00, 2023-01-02 06:00:00]|36.0 |1    |
|[2023-01-02 06:00:00, 2023-01-

In [20]:
from pyspark.sql.functions import col, from_unixtime, unix_timestamp

new_df = spark.sql('select * from aggWindow') \
    .withColumn("start_window", from_unixtime(unix_timestamp(col("window.start")))) \
    .withColumn("end_window", from_unixtime(unix_timestamp(col("window.end")))) \
    .drop("window")

In [21]:
new_df.count()

506

In [22]:
spark.sql('select * from aggWindow').count()

506

In [23]:
new_df.printSchema()

root
 |-- value: float (nullable = true)
 |-- count: long (nullable = false)
 |-- start_window: string (nullable = true)
 |-- end_window: string (nullable = true)



In [24]:
new_df.show()

+-----+-----+-------------------+-------------------+
|value|count|       start_window|         end_window|
+-----+-----+-------------------+-------------------+
| 36.0|    6|2022-12-31 18:00:00|2023-01-01 00:00:00|
| 30.0|    2|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 31.0|    2|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 33.0|    1|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 34.0|    1|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 29.0|    6|2023-01-01 06:00:00|2023-01-01 12:00:00|
| 29.0|    2|2023-01-01 12:00:00|2023-01-01 18:00:00|
| 30.0|    4|2023-01-01 12:00:00|2023-01-01 18:00:00|
| 31.0|    4|2023-01-01 18:00:00|2023-01-02 00:00:00|
| 32.0|    2|2023-01-01 18:00:00|2023-01-02 00:00:00|
| 33.0|    2|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 34.0|    1|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 35.0|    2|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 36.0|    1|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 36.0|    1|2023-01-02 06:00:00|2023-01-02 12:00:00|
| 37.0|    5|2023-01-02 06:0

In [25]:
from pyspark.sql.functions import col, to_timestamp

timestamped_df = new_df.withColumn("start_window", to_timestamp(col("start_window"))) \
    .withColumn("end_window", to_timestamp(col("end_window")))

timestamped_df.printSchema()


root
 |-- value: float (nullable = true)
 |-- count: long (nullable = false)
 |-- start_window: timestamp (nullable = true)
 |-- end_window: timestamp (nullable = true)



In [26]:
timestamped_df.show()

+-----+-----+-------------------+-------------------+
|value|count|       start_window|         end_window|
+-----+-----+-------------------+-------------------+
| 36.0|    6|2022-12-31 18:00:00|2023-01-01 00:00:00|
| 30.0|    2|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 31.0|    2|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 33.0|    1|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 34.0|    1|2023-01-01 00:00:00|2023-01-01 06:00:00|
| 29.0|    6|2023-01-01 06:00:00|2023-01-01 12:00:00|
| 29.0|    2|2023-01-01 12:00:00|2023-01-01 18:00:00|
| 30.0|    4|2023-01-01 12:00:00|2023-01-01 18:00:00|
| 31.0|    4|2023-01-01 18:00:00|2023-01-02 00:00:00|
| 32.0|    2|2023-01-01 18:00:00|2023-01-02 00:00:00|
| 33.0|    2|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 34.0|    1|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 35.0|    2|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 36.0|    1|2023-01-02 00:00:00|2023-01-02 06:00:00|
| 36.0|    1|2023-01-02 06:00:00|2023-01-02 12:00:00|
| 37.0|    5|2023-01-02 06:0

In [27]:
timestamped_df.count()

506

In [28]:


server_name = "jdbc:sqlserver://10.128.0.66"
database_name = "testDB"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "aekanun_pm2022"
username = "SA"
password = "Passw0rd123456"

In [29]:


# Set the connection properties
properties = {
    "user": username,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


# Assuming 'read_result_df' is your DataFrame
timestamped_df.write.jdbc(url=url, table=table_name , mode='overwrite', properties=properties)

